# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [ ]:
import PyPDF2 as pyf
from pathlib import Path

nome = '04 - MGLU_ER_3T20_POR.pdf'
arquivo_pdf = pyf.PdfReader(nome)

for index, pagina in enumerate(arquivo_pdf.pages):
    num_pagina = index + 1

    novo_pdf = pyf.PdfWriter()
    novo_pdf.add_page(pagina)

    with Path(fr'paginas/Arquivo Pagina {num_pagina}.pdf').open(mode='wb') as arquivo:
        novo_pdf.write(arquivo)

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [ ]:
num_paginas = [1, 14, 16]
novo_arquivo = pyf.PdfWriter()

for num in num_paginas:
    pagina_pdf = pyf.PdfReader(f'paginas\Arquivo Pagina {num}.pdf')
    novo_arquivo.add_page(pagina_pdf.pages[0])

with Path(fr'04 - Consolidado.pdf').open(mode='wb') as arquivo:
    novo_arquivo.write(arquivo)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [ ]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append('04 - MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append('04 - MGLU_ER_4T20_POR.pdf')

with Path(fr'04 - Mesclado.pdf').open(mode='wb') as arquivo:
    pdf_mesclado.write(arquivo)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [ ]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append("04 - MGLU_ER_4T20_POR.pdf")
pdf_mesclado.merge(1, 'paginas/Arquivo Pagina 1.pdf')

with Path('04 - Relatorio 2 Trimestres.pdf').open('wb') as arquivo:
    pdf_mesclado.write(arquivo)

- Rodar Página

In [ ]:
arquivo_pdf_original = pyf.PdfReader("04 - MGLU_ER_3T20_POR.pdf")

novo_arquivo = pyf.PdfWriter()

for pagina in arquivo_pdf_original.pages:
    pagina.rotate(90)
    novo_arquivo.add_page(pagina)

with Path('04 - Paginas Rotacionadas.pdf').open('wb') as arquivo:
    novo_arquivo.write(arquivo)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [ ]:
arquivo = pyf.PdfReader("04 - MGLU_ER_3T20_POR.pdf")

qtd_paginas = len(arquivo.pages)

metadados = arquivo.metadata
print(metadados)

In [ ]:
texto_referencia = '| Despesas com Vendas'

for i, pagina in enumerate(arquivo.pages):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        texto_analisar = texto_pagina

print(texto_analisar)

In [ ]:
posicao_inicial = texto_analisar.find(texto_referencia)
posicao_final = texto_analisar.find('|', posicao_inicial + 1)

texto_analisar[posicao_inicial:posicao_final]
print(texto_analisar)



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

In [ ]:
import tabula

tabelas = tabula.read_pdf("04 - MGLU_ER_3T20_POR.pdf", pages=5, encoding='cp1252', guess=False)

df_resultado = tabelas[0]
df_resultado = df_resultado.dropna(how='all', axis=0)
df_resultado = df_resultado.dropna(how='all', axis=1)
df_resultado.columns = df_resultado.iloc[0]
df_resultado = df_resultado.iloc[1:]
df_resultado

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

In [ ]:
tabelas = tabula.read_pdf("04 - MGLU_ER_3T20_POR.pdf", pages=12)
for tabela in tabelas:
    tabela = tabela.dropna(how='all', axis=0)
    tabela = tabela.reset_index(drop=True)
    display(tabela)

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

In [ ]:
df_capital_giro = tabelas[0]
df_capital_giro = df_capital_giro.dropna(how='all', axis=0)
df_capital_giro = df_capital_giro.reset_index(drop=True)

tabelas2 = tabula.read_pdf("04 - MGLU_ER_3T20_POR.pdf", pages=12, lattice=True)
df_capital_giro2 = tabelas2[0]
df_capital_giro2 = df_capital_giro2.dropna(how='all', axis=0)
df_capital_giro2 = df_capital_giro2.dropna(how='all', axis=1)
df_capital_giro2 = df_capital_giro2.reset_index(drop=True)
colunas = df_capital_giro2.iloc[0]
colunas = colunas.dropna()
df_capital_giro.columns = colunas
display(df_capital_giro)

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [4]:
from pikepdf import Pdf, PdfImage

arquivo = Pdf.open("04 - MGLU_ER_3T20_POR.pdf")

for pagina in arquivo.pages:
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imagens/{nome}')

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url